```nasm
;Constant Declaration
yy = 10
bt=100
---
;Variable Declaration
var 1 2 3 4 5
var 3 dup 5 6 dup 10
var "abc"
;this generates a separate variable file
---
;the code section
```

zero = r0
temp = r1
sp = r2
random = r16

In [ ]:
function segments(source) {
	arr = source.split('---');
	if (arr.length!=3) throw "Check segments"
	return {
		constants : arr[0],
		variables : arr[1],
		code : arr[2],
	}
}

In [22]:
function get_constants (source) {
	var constants = new Map()
constants["zero"]="r0"
constants["temp"]="r1"
constants["sp"]="r2"
constants["random"]="r16"
	source = source.split('\n');
	for (var i of source) {
	i=i.replace(/\s|\t/g,'')
	i=i.split(';')[0].split('=')
	if (i.length == 2)constants[i[0]]=i[1]
	}
	return constants
}

In [6]:
get_constants(`
a=1000
b =  3000
`)

Map(0) {
  zero: 'r0',
  temp: 'r1',
  sp: 'r2',
  random: 'r16',
  a: '1000',
  b: '3000'
}


In [21]:
'hi\n  \tthere a'.replace(/\s{2,}/g,' ')

hi there a


In [6]:
function hex29 (n) {
	var ret = (n& 536870911).toString(16).padStart(8,"0")
return ret
}

In [9]:
function memory_preprocess (source) {
	source = source.split('\n');
	var ret =[]
	for (var i of source) {
		var li = i.search(`"`);
		var ri = i.lastIndexOf(`"`);
		if (li==-1) {
			i = i.split(";")[0].trim();
			if (!i.length) continue;
			

		} else {
			 var name  =  i.slice(0,li).trim()
			 var str = i.slice(li+1,ri);
			 var string =[]
			 for (var j=0;j<str.length; j++){
				string.push(hex29(str[j].charCodeAt(0)))
			 }
			 ret.push( {name : name,
							data:string})
		}
	}
	return ret;
}

In [10]:
memory_preprocess(`var "hello"`)

[
  {
    name: 'var',
    string: [ '00000068', '00000065', '0000006c', '0000006c', '0000006f' ]
  }
]


In [9]:
function preprocess(source) {
	var lines = []
	var line =[]
	var temp=""
	var lock =false;
	for(var i=0;i<source.length;i++) {
		var c=source[i]
		if (!lock) 
		switch (c) {
			case ';':
				lock=true;
			case '\n':
				if (temp.length) line.push(temp)
				temp=""
				if (line.length)lines.push(line)
				line=[]
			break
			case ' ':
			case ',':
				if (temp.length) line.push(temp)
				temp=""
				break
			case '\t':
				break
			default:
				temp+=c;
			break
		}
		if (c=='\n') lock=false;
	}
	return lines;
}

In [12]:
preprocess(`
	mov     	         ax,bx;hi
	int 21h
`)

[ [ 'mov', 'ax', 'bx' ], [ 'int', '21h' ] ]


In [ ]:
opcodes =  new Map();
opcodes["nop"]=0
opcodes["add"]=1
opcodes["sub"]=2
opcodes["mul"]=3
opcodes["mulh"]=4
opcodes["div"]=5
opcodes["rem"]=6
opcodes["not"]=7
opcodes["and"]=8
opcodes["or"]=9
opcodes["xor"]=10
opcodes["shl"]=11
opcodes["shr"]=12
opcodes["ror"]=13
opcodes["rol"]=14
opcodes["ashl"]=15
opcodes["addi"]=17
opcodes["subi"]=18
opcodes["muli"]=19
opcodes["mulhi"]=20
opcodes["divi"]=21
opcodes["remi"]=22
opcodes["lui"]=23
opcodes["andi"]=24
opcodes["ori"]=25
opcodes["xori"]=26
opcodes["shli"]=27
opcodes["shri"]=28
opcodes["rori"]=29
opcodes["roli"]=30
opcodes["ashli"]=31
opcodes["jr"]=32
opcodes["bg"]=33
opcodes["be"]=34
opcodes["bge"]=35
opcodes["bl"]=36
opcodes["bne"]=37
opcodes["ble"]=38
opcodes["jal"]=39
opcodes["load"]=40
opcodes["loadi"]=41
opcodes["store"]=42
opcodes["storei"]=43
opcodes["in"]=44
opcodes["ini"]=45
opcodes["out"]=46
opcodes["outi"]=47
opcodes["hlt"]=63
registers = new Map();
registers["r0"]=0
registers["r1"]=1
registers["r2"]=2
registers["r3"]=3
registers["r4"]=4
registers["r5"]=5
registers["r6"]=6
registers["r7"]=7
registers["r8"]=8
registers["r9"]=9
registers["r10"]=10
registers["r11"]=11
registers["r12"]=12
registers["r13"]=13
registers["r14"]=14
registers["r15"]=15
registers["r16"]=16
registers["r17"]=17
registers["r18"]=18
registers["r19"]=19
registers["r20"]=20
registers["r21"]=21
registers["r22"]=22
registers["r23"]=23
registers["r24"]=24
registers["r25"]=25
registers["r26"]=26
registers["r27"]=27
registers["r28"]=28
registers["r29"]=29
registers["r30"]=30
registers["r31"]=31

In [ ]:
function specify (line) {

}

In [ ]:
function expand(line) { //do before push
	switch(line[0]) {
		case "mov":
			break;
		case "push":
			
	}
}